In [2]:
import pandas as pd
from dataclasses import asdict
from backdoors import data, patterns, poison, utils, checkpoint_dir
from backdoors.data import load_cifar10, Data
import numpy as np
import matplotlib.pyplot as plt
import einops
import jax
from jax import numpy as jnp
from flax.training import train_state
import optax
import chex
import flax
from flax import linen as nn
from typing import Tuple
import matplotlib.pyplot as plt
import sympy
from jax import random
from collections import namedtuple
from time import time
rng = random.PRNGKey(0)

import orbax.checkpoint
import os
from pathlib import Path

from backdoors import module_path, checkpoint_dir, train
from backdoors.train import TrainState, Metrics, model, tx, \
    accuracy, train_step, init_train_state
from backdoors.models import CNN
from meta_transformer.data import load_pair_of_models, data_iterator
from tqdm import tqdm


checkpointer = orbax.checkpoint.PyTreeCheckpointer()

BATCH_SIZE = 64
NUM_EPOCHS = 5

#train_data, _ = load_cifar10()
cifar10_train, cifar10_test = load_cifar10()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = CNN()
params = model.init(rng, jnp.ones((1, 32, 32, 3)))["params"]
jax.tree_map(lambda x: x.shape, params)

{'Conv_0': {'bias': (32,), 'kernel': (3, 3, 3, 32)},
 'Conv_1': {'bias': (32,), 'kernel': (3, 3, 32, 32)},
 'Conv_2': {'bias': (64,), 'kernel': (3, 3, 32, 64)},
 'Conv_3': {'bias': (64,), 'kernel': (3, 3, 64, 64)},
 'Conv_4': {'bias': (128,), 'kernel': (3, 3, 64, 128)},
 'Conv_5': {'bias': (128,), 'kernel': (3, 3, 128, 128)},
 'Dense_6': {'bias': (10,), 'kernel': (128, 10)}}

In [6]:
model.apply({"params": params}, jnp.ones((1, 32, 32, 3)))

Array([[ 0.17627773, -0.00739185,  0.37206712,  0.02501557,  0.5720889 ,
        -0.12718616, -0.00155175,  0.11558752, -0.11277112, -0.16591665]],      dtype=float32)

In [14]:
z = jax.xla_computation(model.apply)({"params": params}, jnp.ones((1, 32, 32, 3)))

In [16]:
z[1]

ShapeDtypeStruct(shape=(1, 10), dtype=float32)

In [13]:
with open("test", "w") as f:
    f.write(z.as_hlo_dot_graph())